In [1]:

####################################################################################################
####################################################################################################

import sys
import os

import datetime
import matplotlib.pyplot as plt
import numpy as np
import numba
import pandas as pd
import pprint

from IPython.display import clear_output
from IPython.display import display

sys.path.append(os.path.abspath("../"))
sys.path.append(os.path.abspath("../pax"))

from pax_utils import event_utils
from pax_utils import file_utils
from pax_utils import interaction_utils
from pax_utils import s1s2_utils
from pax_utils import waveform_utils
from pax_utils import waveform_raw_utils

from pax.configuration import load_configuration

pp = pprint.PrettyPrinter(depth=2)



In [ ]:

####################################################################################################
####################################################################################################

f = plt.figure(figsize=(30, 15))

def processPklEvents(zipfilename):

    ################################################################################################
    ################################################################################################
    
    lst_pkl_files  = file_utils.getListOfPklFilesFromZipFile(zipfilename, nEventsPerFile)

    
    ################################################################################################
    ################################################################################################
    
    intr0  = None
    sum_wf = None
    event  = None

    df_zip_merged = pd.DataFrame()
    
        
    ################################################################################################
    ################################################################################################
    
    for iPklFile in range(0, len(lst_pkl_files)):
    
        ############################################################################################
        ############################################################################################

        print("File: '" + str(zipfilename) + "'")
        print("   event_number: " + str(iPklFile))
        #clear_output(wait=True)
        
        
        ############################################################################################
        ############################################################################################

        jsonfilename = os.path.dirname(zipfilename) + '/pax_info.json'
        print("   CFG: '" + str(jsonfilename) + "'")
        
        cfg = event_utils.getConfig(jsonfilename)
        
        
        ############################################################################################
        ############################################################################################
        
        pklfilename  = lst_pkl_files[iPklFile]
        event        = file_utils.getPaxEventFromPklFileInZipArchive(zipfilename, pklfilename)

        
        ############################################################################################
        ############################################################################################
        
        interactions  = event.interactions
        nInteractions = len(interactions)
        
        if (nInteractions != 1): continue
        
        
        ############################################################################################
        ############################################################################################

        intr0 = None
        
        if (len(interactions) > 0):
            
            intr0 = interactions[0]
            
            
        ############################################################################################
        ############################################################################################
        
        df_pkl_event  = event_utils.getEventDataFrameFromEvent(event)
        df_pkl_intr   = interaction_utils.getInteractionDataFrameFromEvent(event)
        df_pkl_s2s    = s1s2_utils.getS2integralsDataFrame(event, 127)
        df_pkl_merged = df_pkl_event.merge(df_pkl_intr).merge(df_pkl_s2s)
        df_zip_merged = df_zip_merged.append(df_pkl_merged)
        
              
        #############################################################################################
        # Individual Waveforms
        #############################################################################################

        #df_wfs = waveform_utils.getS2waveformsAllChannels(event, cfg)
        #print(df_wfs.shape)
        #display(df_wfs[0:1][:])
        
        
        ############################################################################################
        # S2 Window from Event
        ############################################################################################
        
        left  = event.main_s2.left
        right = event.main_s2.right
        
    
        ############################################################################################
        # Raw Waveforms
        ############################################################################################
    
        arr_summed_waveform_raw_evt = waveform_utils.getSummedWaveformFromEvent(event, 'tpc_raw')[left:right]
        df_channels_waveforms_raw   = waveform_raw_utils.getChannelWaveformsDataFrameRaw(event, cfg)
        arr_summed_waveform_raw_df  = waveform_raw_utils.getSummedWaveformFromDataFrame(df_channels_waveforms_raw, event)
        arr_summed_waveform_raw_df  = arr_summed_waveform_raw_df[left:right]
        
        f = plt.figure(figsize=(10, 5))
        plt.plot(arr_summed_waveform_raw_evt, color='b', label='Raw Summed Waveform from Event') 
        plt.plot(arr_summed_waveform_raw_df , color='r', label='Raw Summed Waveform by Hand DF') 
        plt.legend(fontsize=18)
        
        eq_raw = waveform_raw_utils.compareWaveforms(arr_summed_waveform_raw_evt, arr_summed_waveform_raw_df)
                                                                                      
        if (not eq_raw):
            
            raise Exception("Error! RAW Waveforms summed over channels by hand differ from event (DF).")
            return
      
        continue
        
        
        

        ############################################################################################
        # Summed Waveform
        ############################################################################################
        
        evt_summed_waveform_tpc = waveform_utils.getSummedWaveformFromEvent(event, 'tpc')[left:right]
        tst_summed_waveform_tpc = waveform_raw_utils.getSummedWaveform(event, cfg)[left:right]

        f = plt.figure(figsize=(10, 5))
        plt.plot(evt_summed_waveform_tpc, color='b', label='TPC Summed Waveform from Event') 
        plt.plot(tst_summed_waveform_tpc, color='g', label='TPC Summed Waveform from Test') 
        plt.legend(fontsize=18)
        
        eq_tpc = waveform_raw_utils.compareWaveforms(evt_summed_waveform_tpc, tst_summed_waveform_tpc)
        
        #continue
        
        
        ############################################################################################
        # Top Summed Waveform
        ############################################################################################
    
        evt_summed_waveform_top   = waveform_utils.getSummedWaveformFromEvent(event, 'tpc_top')[left:right]
        tst_summed_waveform_top   = waveform_raw_utils.getSummedWaveform(event, cfg, 126)[left:right]
        #df_channels_waveforms_top = waveform_raw_utils.getChannelWaveformsDataFrameTop(event, cfg)
        #arr_summed_waveform_top   = waveform_raw_utils.getSummedWaveformFromDataFrame(df_channels_waveforms_top, event, False)
        #arr_summed_waveform_top   = arr_summed_waveform_top[left:right]
        
        f = plt.figure(figsize=(10, 5))
        plt.plot(evt_summed_waveform_top, color='b', label='Top Summed Waveform from Event') 
        plt.plot(tst_summed_waveform_top, color='g', label='Top Summed Waveform from Test') 
        #plt.plot(arr_summed_waveform_top, color='r', label='Top Summed Waveform By Hand'   ) 
        
        eq_top = waveform_raw_utils.compareWaveforms(evt_summed_waveform_top, tst_summed_waveform_top)
        
        
           
        ############################################################################################
        ############################################################################################
        
        #eq0 = waveform_raw_utils.compareWaveforms(evt_summed_waveform_top, tst_summed_waveform_top)
        
        #if (not eq0): raise Exception("Error! RAW Waveforms summed over channels by hand differ from event (ARR).")
            
            
            
        
        #############################################################################################
        ## Reproduce Waveform from Event with PAX
        #############################################################################################
        #
        #arr_sum_wf2 = waveform_utils.SumWaveformPAX(event)
        #arr_sum_wf2 = arr_sum_wf2[left:right]
        #arr_sum_wf2 *= 0.5
        #plt.plot(arr_sum_wf2, color='g', label='Rerunning PAX')
        
        
        ##############################################################################################
        ##############################################################################################
        
        #arr_sum_wf3 = waveform_utils.SumWaveformTestCustom(event, cfg, -1)
        #arr_sum_wf3 = arr_sum_wf3[left:right]
        ##arr_sum_wf3 *= 0.5
        #
        #plt.plot(arr_sum_wf3, color='r', label='Simplified')

           
            
        #############################################################################################
        # Individual Waveforms
        #############################################################################################
        
        #arr_sum_wf3_chan0 = waveform_utils.SumWaveformTestCustom(event, cfg, 0)
        #arr_sum_wf3_chan9 = waveform_utils.SumWaveformTestCustom(event, cfg, 9)

        #print(arr_sum_wf3_chan0.shape)
        #print(arr_sum_wf3_chan9.shape)
  

        #############################################################################################
        # Check Summed Waveforms
        #############################################################################################

        #arr_sum_wf4 = waveform_utils.getSummedWaveform(df_wfs, cfg, event)[left:right]
        #arr_sum_wf4 *= 0.5

        #plt.plot(arr_sum_wf4, color='k', label='Waveforms Summed By Hand')
        

        
        
        ############################################################################################
        ############################################################################################

        plt.legend(fontsize=18)
        #plt.xlim([0, 500])
        #plt.savefig('summed_waveform_overlay_shifted.png')        
        
               
                
        ############################################################################################
        # Sanity
        ############################################################################################

        event_sum_wf  = np.sum(evt_summed_waveform_top)
        event_sum_s2s = np.sum(df_pkl_s2s.drop(columns=['event_number']).as_matrix(), axis=1) 
        
        print()
        print("Event Sum Waveform: " + str(event_sum_wf))
        print("Event Sum S2's:     " + str(event_sum_s2s))

       
    
        #############################################################################################
        #############################################################################################

        assert(18 == len(event.get_sum_waveform_names()))
        
        #for name in event.get_sum_waveform_names(): print(name)

            
        ############################################################################################
        ############################################################################################
    
        continue
            
    
    ################################################################################################
    ################################################################################################
    
    #plt.legend()
    
    df_zip_merged.reset_index(inplace=True, drop=True)
       
    return df_zip_merged, event, intr0, sum_wf

    

In [ ]:

####################################################################################################
####################################################################################################

zipfilename    = '../pax_run/pax_output/Mar25/instructions_000000/XENON1T-0-000000000-000000999-000001000.zip'
nEventsPerFile = 2


df_zip_merged, event, intr, wf_sum = processPklEvents(zipfilename)


    